# Image Generator


In [254]:
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib import pyplot
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
import glob, os

#os.chdir("..")
print(os.getcwd())

#os.listdir(os.getcwd())
#os.chdir("My Drive/Colab Notebooks/images")
#print(os.getcwd())


/Users/lusy/Desktop/MasterThesis


In [257]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        #rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        #zoom_range=0.2,
        #horizontal_flip=True,
        fill_mode='nearest')


for file in glob.glob("weird images/NeckerCubeDrawings/healthy/" + "*.png"):
    img = load_img(file)  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='weird images/NeckerCubeDrawings/healthy', save_format='png'):
        i += 1
        if i > 15:
            break 
            
for file in glob.glob("weird images/NeckerCubeDrawings/PD/" + "*.png"):
    img = load_img(file)  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='weird images/NeckerCubeDrawings/healthy', save_format='png'):
        i += 1
        if i > 10:
            break 

In [258]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
import pickle


file_list = []
class_list = []

DATADIR = "weird images/NeckerCubeDrawings"

# All the categories you want your neural network to detect
CATEGORIES = ["healthy", "PD"]

# The size of the images that your neural network will use
IMG_SIZE = 150

# Checking or all images in the data folder
for category in CATEGORIES :
	path = os.path.join(DATADIR, category)
	for img in os.listdir(path):
		img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)

training_data = []

def create_training_data():
	for category in CATEGORIES :
		path = os.path.join(DATADIR, category)
		class_num = CATEGORIES.index(category)
		for img in os.listdir(path):
			try :
				img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
				new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
				training_data.append([new_array, class_num])
			except Exception as e:
				pass

create_training_data() # contains training_data with images and classes

random.shuffle(training_data) #shufles the data so it's not control and patient

X = [] #features
y = [] #labels
#y = np.array(y)

for features, label in training_data:
	X.append(features)
	y.append(label)



"""""""""
# Creating the files containing all the information about your model
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)


#y=np.array(y)
#y.shape

"""
len(X),len(y)

(2993, 2993)

In [259]:
def split_datatest(x,y):

    # train and test
    X_train = X[:int(0.9*int(len(X)))]
    y_train = y[:int(0.9*int(len(X)))]
    X_test = X[int(0.9*int(len(X))):]
    y_test = y[int(0.9*int(len(y))):]
    
    # validation
    X_train = X_train[:int(0.8*int(len(X_train)))]
    y_train = y_train[:int(0.8*int(len(y_train)))]
    
    X_val = X_train[int(0.8*int(len(X_train))):]
    y_val = y_train[int(0.8*int(len(y_train))):]
    
    return X_train, y_train, X_test, y_test, X_val, y_val

#len(X_val), len(y_val)


In [260]:
def create_validation(X_train,y_train):
    X_train = X_train[:int(0.8*int(len(X_train)))]
    y_train = y_train[:int(0.8*int(len(y_train)))]
    
    X_val = X_train[int(0.8*int(len(X_train))):]
    y_val = y_train[int(0.8*int(len(y_train))):]
    
    return X_train, y_train, X_val, y_val

In [261]:

X_train, y_train, X_test, y_test, X_val, y_val = split_datatest(X,y)

print(len(X_val), len(y_val),len(X_train), len(y_train), len(X_test), len(y_test))

431 431 2154 2154 300 300


In [262]:
def prepare_dataset(X_train, y_train, X_test, y_test, X_val, y_val):

    # reshape training sets to prepare for CNN
    X_train = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    y_train = np.array(y_train)
    
    X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    y_test = np.array(y_test)
    
    X_val = np.array(X_val).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    y_val = np.array(y_val)
    
    # one hot encoding of pixels
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    y_val = to_categorical(y_val)
    
    train_norm = X_train.astype('float32')
    test_norm = X_test.astype('float32')
    val_norm = X_val.astype('float32')

    # normalize to range 0-1
    X_train = train_norm / 255.0
    X_test = test_norm / 255.0
    X_val = val_norm / 255.0
    # return dataset
    
    return X_train, y_train, X_test, y_test, X_val, y_val

In [265]:
# test harness for evaluating models on the cifar10 dataset
import sys
from matplotlib import pyplot
from keras.datasets import cifar10, mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import SGD

 
# define cnn model
def define_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(150, 150, 1)))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.2))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.3))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.4))
  model.add(Flatten())
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dropout(0.5))
  model.add(Dense(2, activation='softmax'))
  # compile model
  opt = SGD(lr=0.001, momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  print("Model defined")
  return model
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    #pyplot.show()
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()
    
 
# run the test harness for evaluating a model
def run_test_harness(x,y):
    
    #X_train, y_train, X_test, y_test = create_test(X,y)
    
    #X_train, y_train, X_val, y_val = create_validation(X_train,y_train)
    
    X_train, y_train, X_test, y_test, X_val, y_val = split_dataset(X,y)
    
    X_train, y_train, X_test, y_test, X_val, y_val = prepare_dataset(X_train, y_train, X_test, y_test, X_val, y_val)
    
    print(len(X_val), len(y_val))
          
    model = define_model()
    # fit model
    history = model.fit(X_train, y_train, epochs=10, batch_size=24, validation_data=(X_val, y_val), verbose=0)
    # evaluate model
    _, acc = model.evaluate(X_test, y_test, verbose=0)
    print('Test Accuracy: %.3f' % (acc * 100.0))
    # learning curves
    summarize_diagnostics(history)


In [ ]:
run_test_harness(X,y)

431 431
Model defined
